In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import *
import numpy as np
import pandas as pd
import sys, os
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from GCFNN import GCFNN
import torch.nn.functional as F
from numpy.random import randint

In [2]:
def concatenated_tensor(X_set, M):
    concatenated_tensor_list = []
    # 遍历索引范围为 0 到 M-1 的值
    for i in range(M.shape[1]):
        # 拼接 X_set[i]
        concatenated_tensor_list.append(X_set[i])
        ## 将 adj_list[i] 转换为密集张量并拼接
        #adj_dense_tensor = adj_list[i].to_dense()
        #concatenated_tensor_list.append(adj_dense_tensor)
        
    # 将 tr_M 转换为张量并拼接
    M_tensor = torch.from_numpy(M)
    concatenated_tensor_list.append(M_tensor)
    # 沿着 dim=1 拼接所有张量
    concatenated_tensor = torch.cat(concatenated_tensor_list, dim=1)
    return concatenated_tensor

In [3]:
# read data
meta_data=pd.read_csv('meta.csv',index_col=0)
metabolomics=pd.read_csv('metabolomics.csv',index_col=0).fillna(0)
metagenomics=pd.read_csv('metagenomics.csv',index_col=0)
data_class={'nonIBD':0,'CD':1,'UC':1}
meta_data['diagnosis']=meta_data['diagnosis'].map(data_class)

x1=metagenomics
x2=metabolomics
print('输入各组学数据表：',x1.shape,x2.shape,meta_data.shape)

#重构数据表和标签表
X1=pd.concat([x1,x2],axis=1).iloc[:,:x1.shape[1]]
X2=pd.concat([x1,x2],axis=1).iloc[:,x1.shape[1]:x1.shape[1]+x2.shape[1]]
index_new=X1.index
print(index_new)
missing_rate=(len(x1)+len(x2))/(len(index_new)*2)
print('数据完整率missing_rate :',missing_rate)
                                        
meta_data=meta_data.reindex(index_new)
X1=array(X1.reindex(index_new))
X2=array(X2.reindex(index_new))
print('集合所有样本重构数据表：',X1.shape,X2.shape,meta_data.shape)
X_set=[X1,X2]
M        = len(X_set)
Mask     = np.ones([np.shape(X_set[0])[0], M])
#生成表示组学缺失情况的掩码矩阵，将缺失组学部分填补为0
for m_idx in range(M):
    Mask[np.isnan(X_set[m_idx]).all(axis=1), m_idx] = 0
    #X_set[m_idx][Mask[:, m_idx] == 0] = np.mean(X_set[m_idx][Mask[:, m_idx] == 1], axis=0)
    X_set[m_idx][Mask[:, m_idx] == 0] = 0
    
#对宏基因组数据预处理
def log(base,x):
    return np.log(x)/np.log(base)
X_set[0]=log(2,2*X_set[0]+0.00001)

#print(Mask)
#print(X_set)
Y = meta_data['diagnosis'].tolist()
Y = np.array(Y)
Y_onehot = np.zeros((Y.shape[0], 2))
Y_onehot[np.arange(Y.shape[0]), Y] = 1
print('标签转换为one-hot编码：', Y_onehot.shape, np.sum(Y_onehot[:, 0]))

输入各组学数据表： (1638, 578) (546, 596) (2891, 1)
Index(['PSM7J1BF', 'CSM7KORK', 'HSM7J4HS', 'MSM9VZNL', 'MSMB4LYB', 'MSM6J2PM',
       'MSMA26ET', 'CSM79HMN', 'CSM67UB1', 'HSM7J4IS',
       ...
       'PSM7J134', 'HSM5FZBJ', 'CSM79HK1', 'CSM5MCTZ', 'CSM9X21P', 'CSM5MCXF',
       'CSM67U9P', 'CSM7KOPQ', 'HSMA33LV', 'MSM5LLFK'],
      dtype='object', length=1796)
数据完整率missing_rate : 0.6080178173719376
集合所有样本重构数据表： (1796, 578) (1796, 596) (1796, 1)
标签转换为one-hot编码： (1796, 2) 459.0


In [4]:
import shap
import matplotlib.pyplot as plt
#os.makedirs('figure', exist_ok=True)
folder_path = 'model_shap'
cv = StratifiedKFold(n_splits=5,shuffle = True,random_state = 50)

new_mg_names = ['s' + col.split('|s', 1)[1] for col in metagenomics.columns]
combined_columns = new_mg_names + metabolomics.columns.to_list()
for m in range(M):
    combined_columns.append('mask'+str(m))
train_index, test_index = next(cv.split(X_set[0],np.argmax(Y_onehot,axis=1)))
tr_X_set, te_X_set, va_X_set = {}, {}, {}
for m in range(len(X_set)):
    tr_X_set[m],tr_Y_onehot,tr_M = X_set[m][train_index],Y_onehot[train_index],Mask[train_index]
    te_X_set[m],te_Y_onehot,te_M = X_set[m][test_index],Y_onehot[test_index],Mask[test_index]
    #归一化
    #print(te_Y_onehot)
def Normalize(data):
    """
    :param data:Input data
    :return:normalized data
    """
    mean = np.mean(data)
    mx = np.max(data)
    mn = np.min(data)
    return mean, mx, mn

for m in range(M):
    mean, mx, mn = Normalize(tr_X_set[m])
    tr_X_set[m] = (tr_X_set[m] - mean) / (mx - mn)
    te_X_set[m] = (te_X_set[m] - mean) / (mx - mn)
    
for m in range(M):
    tr_X_set[m] = torch.from_numpy(tr_X_set[m])
    te_X_set[m] = torch.from_numpy(te_X_set[m])
tr_Y_onehot = torch.from_numpy(tr_Y_onehot)
te_Y_onehot = torch.from_numpy(te_Y_onehot)

#tansform
tr_input = concatenated_tensor(tr_X_set, tr_M)
te_input = concatenated_tensor(te_X_set, te_M)   
model_path = os.path.join(folder_path, 'model_'+str(0)+'.pth')
model_test = torch.load(model_path)

if str(model_test.device) == 'cuda':
    tr_input = tr_input.cuda()
    te_input = te_input.cuda()
    tr_Y_onehot = tr_Y_onehot.cuda()
    te_Y_onehot = te_Y_onehot.cuda()
model_test.predict_test(te_input, te_Y_onehot)

# select a set of background examples to take an expectation over
#background = tr_input[np.random.choice(tr_input.shape[0], 100, replace=False)]


# explain predictions of the model on four images
e = shap.DeepExplainer(model_test, tr_input)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)

Test F1: 0.9760
Test ACC: 0.9639
Test AUC: 0.9858
you are test train_data


In [5]:
#shap_values = e.shap_values(te_input)
#print(te_input.shape)
#shap.summary_plot(shap_values[0][:,:578], te_input[:,:578], max_display = 15, show = False)
#plt.show()
#shap.summary_plot(shap_values[0][:,578:-2], te_input[:,578:-2], max_display = 15, show = False)
#plt.show()

In [6]:
merged_input = torch.cat((tr_input, te_input), dim=0)
merged_input_index = index_new[train_index].tolist()+index_new[test_index].tolist()
condition = (merged_input[:, -2:] == 1).all(axis=1)
co_input = merged_input[condition]
co_input_index = [idx for idx, cond in zip(merged_input_index, condition) if cond]

In [7]:
shap_values = e.shap_values(co_input)
print(co_input.shape)
pd.DataFrame(shap_values[0],columns = combined_columns, index = co_input_index).to_csv("co_shap_values.csv", index=True)

RuntimeError: CUDA out of memory. Tried to allocate 6.91 GiB (GPU 0; 31.75 GiB total capacity; 22.09 GiB already allocated; 1.33 GiB free; 29.04 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
#不运行上段代码，直接读取已有shap数据
shap_values={}
co_shap_values = pd.read_csv("co_shap_values.csv", index_col=0)
shap_values[0] = co_shap_values.values

In [ ]:
new_mg_names = ['s' + col.split('|s', 1)[1] for col in metagenomics.columns]
combined_columns = new_mg_names + metabolomics.columns.to_list()
for m in range(M):
    combined_columns.append('mask'+str(m))
    
shap.initjs()
shap.summary_plot(shap_values[0][:,:578], pd.DataFrame(array(co_input.cpu()),columns = combined_columns).iloc[:,:578], max_display = 20, show = False)
plt.savefig('keymg_20.svg')
plt.show()
shap.summary_plot(shap_values[0][:,578:-2], pd.DataFrame(array(co_input.cpu()),columns = combined_columns).iloc[:,578:-2], max_display = 20, show = False)
plt.savefig('keymb_20.svg')
plt.show()

In [ ]:
# 计算平均的绝对值
mean_abs_shap_values = np.mean(np.abs(shap_values[0]), axis=0)

# 获取特征排名
feature_ranking = np.argsort(-mean_abs_shap_values)

#按排名打印特征
mg_fe_all = open('co_mg_important_fe_all.txt',mode='a')
mb_fe_all = open('co_mb_important_fe_all.txt',mode='a')
combined_columns = metagenomics.columns.to_list() + metabolomics.columns.to_list()
combined_columns[:578] = [col.split('|s__', 1)[1] for col in combined_columns[:578]]
for j in range(len(feature_ranking)):
    if feature_ranking[j] < 578:
        mg_fe_all.writelines(str(feature_ranking[j])+'\t'+combined_columns[feature_ranking[j]]+'\n')
    elif feature_ranking[j] <578+596:
        mb_fe_all.writelines(str(feature_ranking[j]-578)+'\t'+combined_columns[feature_ranking[j]]+'\n')
mg_fe_all.close()
mb_fe_all.close()

In [ ]:
#取shap排名前二十的菌和代谢物用shap值作斯皮尔曼相关性分析，画聚类热力图
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# 加载CSV文件
df = pd.read_csv('co_shap_values.csv', index_col=0)

# 选择"label"列值为-1的行
#selected_rows = df[df['label'] == -1]
selected_rows = df
print(selected_rows.shape)

# 提取特征列
mg_selected_columns = [105,384,431,355,92,106,569,73,281,17,315,298,319,391,71,152,26,157,159,70]
mb_selected_columns = [541,439,462,578,438,593,543,545,515,580,337,526,554,550,441,16,575,413,436,450]
mb_selected_columns = [578+index for index in mb_selected_columns]
mg_features = selected_rows.iloc[:, mg_selected_columns]
mb_features = selected_rows.iloc[:, mb_selected_columns]

# 获取新列名的列表
new_mg_names = ['|s' + col.split('|s', 1)[1] for col in mg_features.columns]

# 重命名列名
mg_features.columns = new_mg_names

# 初始化相关性系数矩阵
num_mg_features = len(mg_features.columns)
num_mb_features = len(mb_features.columns)
correlation_matrix_spearman = np.zeros((num_mg_features, num_mb_features))
p_value_matrix_spearman = np.zeros((num_mg_features, num_mb_features))

# 计算不同相关性系数
for i, feature1 in enumerate(mg_features.columns):
    for j, feature2 in enumerate(mb_features.columns):
        correlation_spearman, p_value_spearman = stats.spearmanr(mg_features[feature1], mb_features[feature2])
        correlation_matrix_spearman[i, j] = correlation_spearman
        p_value_matrix_spearman[i, j] = p_value_spearman
        
# 设置p值阈值，例如，p值小于0.05
p_value_threshold = 0.05

# 根据p值修改热力图颜色块
correlation_matrix_modified = np.where(p_value_matrix_spearman < p_value_threshold, correlation_matrix_spearman, np.nan)

#增加shap正负性
mb_shap = np.array([1,-1,-1,-1,1,-1,-1,-1,1,1,1,-1,-1,1,-1,-1,1,1,-1,1]) 
correlation_matrix_modified = np.vstack([correlation_matrix_modified, mb_shap])
mg_shap = np.array([1,1,1,1,1,-1,-1,-1,-1,1,1,1,-1,1,1,1,-1,1,-1,1,np.nan]) 
mg_shap = mg_shap.reshape(-1, 1)  # 将一维数组转换为二维列向量
correlation_matrix_modified = np.hstack([correlation_matrix_modified, mg_shap])

# Create a heatmap with modified colors based on p-value
mg_label = mg_features.columns.tolist()
mg_label.append('SHAP_to_Healthy')
mb_label = mb_features.columns.tolist()
mb_label.append('SHAP_to_Healthy')

plt.figure(figsize=(18, 12))
sns.heatmap(correlation_matrix_modified, annot=True, fmt=".2f", cmap='coolwarm', xticklabels=mb_label, yticklabels=mg_label,vmin=-1,vmax=1)
plt.title('Modified Spearman Correlation Heatmap based on p-value')
plt.show()

In [ ]:
# 将NaN值替换为一个正数（例如0）
correlation_matrix_modified[np.isnan(correlation_matrix_modified)] = 0

# 绘制聚类图
plt.figure(figsize=(18, 12))
sns.clustermap(correlation_matrix_modified, cmap='coolwarm', xticklabels=mb_label, yticklabels=mg_label,vmin=-1,vmax=1)
plt.title('Clustered Spearman Correlation Heatmap based on p-value')
plt.show()

In [ ]:
#斯皮尔曼相关性

from scipy.stats import spearmanr
# 提取shap_values中的相关列
selected_columns = [105,384,431,355,92,106,569,73,281,17,315,298,319,391,71,152,26,157,159,70]
selected_values = shap_values[0][:, selected_columns]

# 提取第579列到倒数第三列的数据
reference_values = shap_values[0][:, 578:-2]
# 创建一个DataFrame来存储相关性结果
correlation_df = pd.DataFrame()

# 计算每一列与参考列的斯皮尔曼相关性
for i in range(selected_values.shape[1]):
    selected_col = selected_values[:, i]
    correlations = [spearmanr(selected_col, reference_col).correlation for reference_col in reference_values.T]
    correlation_df[f"Column {selected_columns[i]}"] = correlations
    
# 将相关性表保存为CSV文件
correlation_df = correlation_df.T
new_mg_names = [col.split('|s__', 1)[1] for col in metagenomics.columns]
correlation_df.index = [new_mg_names[i] for i in selected_columns]
correlation_df.columns = metabolomics.columns
correlation_df.to_csv("spearmanr_correlation_table.csv", index=True)

In [ ]:
##皮尔森相关性
#
#from scipy.stats import pearsonr 
## 提取shap_values中的相关列
#selected_columns = [105,384,431,355,92,106,569,73,281,17]
#selected_values = shap_values[0][:, selected_columns]
#
## 提取第579列到倒数第四列的数据（第579列到倒数第三列是n-4到n-1）
#reference_values = shap_values[0][:, 578:-2]
##pd.DataFrame(shap_values[0]).to_csv("co_shap_values.csv", index=True)
## 创建一个DataFrame来存储相关性结果
#correlation_df = pd.DataFrame()
#
## 计算每一列与参考列的斯皮尔曼相关性
#for i in range(selected_values.shape[1]):
#    selected_col = selected_values[:, i]
#    correlations = [pearsonr(selected_col, reference_col)[0] for reference_col in reference_values.T]
#    correlation_df[f"Column {selected_columns[i]}"] = correlations
#    
## 将相关性表保存为CSV文件
#correlation_df = correlation_df.T
#new_mg_names = [col.split('|s__', 1)[1] for col in metagenomics.columns]
#correlation_df.index = [new_mg_names[i] for i in selected_columns]
#correlation_df.columns = metabolomics.columns
#correlation_df.to_csv("pearsonr_correlation_table.csv", index=True)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# 假设correlation_df是一个10*578的DataFrame，行名为父节点的名称
# 假设每个行名与在该行中绝对值排名前二十的列名相连接
# 假设您已经选择了前二十个相关性最高的列

# 创建一个有向图
G = nx.DiGraph()

# 添加根节点'IBD_to_normal'
G.add_node('IBD_to_normal')

# 添加父节点（行名），并连接到根节点
for parent_node in correlation_df.index:
    G.add_node(parent_node)
    for correlation_value in [1,1,1,1,1,-1,-1,-1,-1,1,1,1,-1,1,1,1,-1,1,-1,1]:
        G.add_edge('IBD_to_normal', parent_node, weight = correlation_value)

# 添加子节点（前二十的列名）并连接到父节点，权重为相关性值
for parent_node in correlation_df.index:
    top_20_correlations = correlation_df.loc[parent_node].abs().nlargest(20)
    top_20_correlations = correlation_df.loc[parent_node,top_20_correlations.index]
    for child_node, correlation_value in top_20_correlations.iteritems():
        G.add_node(child_node)
        G.add_edge(parent_node, child_node, weight=correlation_value)

# 调整节点位置以更好显示根节点
pos = nx.spring_layout(G, seed=42, k=0.25, iterations=50)  # 调整k和iterations以更好显示根节点

# 获取边的权重
weights = [(u, v, f"{G[u][v].get('weight', 1.0):.2f}") for u, v in G.edges()]  # 保留小数点后两位并保留正负符号

# 绘制网络图，包括权值标签
plt.figure(figsize=(12, 12))
nx.draw_networkx(G, pos, labels=None, node_size=1000, node_color='lightblue', font_size=10, font_color='black',
                 edge_color='gray', width=1.0, alpha=0.7, with_labels=True, font_weight='bold', arrows=True)

# 添加权值标签
#edge_labels = {(u, v): label for u, v, label in weights}
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# 添加节点标签
node_labels = {node: node for node in G.nodes()}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, font_color='black')

plt.title("Network Graph of Correlations")
plt.axis('off')
plt.show()

In [ ]:
import pandas as pd

key = 0.5

# 创建 'unweight.glasso_edge' 文件
edge_file = 'unweight.glasso_edge' + str(key) + '.tsv'
node_weight_mapping = {}
with open(edge_file, 'w') as f:
    # 写入具有新列名的表头
    f.write("id\tsource\ttarget\n")

    # 初始化链接计数器
    link_counter = 1

    # 初始化用于存储已写入的节点的集合
    written_nodes = set()

    # 写入相关性大于key的边，并逐渐增加链接名称
    for idx in correlation_df.index:
        written_nodes.add(idx)
        node_weight_mapping[idx] = 100
        for col in correlation_df.columns:
            correlation_value = correlation_df.at[idx, col]
            if correlation_value >= key:
                print(idx,col)
                # 写入边的信息
                f.write("link_{}\t{}\t{}\n".format(link_counter, idx, col))
                link_counter += 1

                # 将节点添加到已写入的节点集合中
                written_nodes.add(col)

                # 创建节点到权值的映射
                node_weight_mapping[col] = 10

# 创建 'unweight.glasso_node' 文件
node_file = 'unweight.glasso_node' + str(key) + '.tsv'
with open(node_file, 'w') as f:
    # 写入具有新列名的表头
    f.write("id\tlabel\n")

    # 写入已写入的节点以及根据映射分配的权值
    for node in written_nodes:
        weight = node_weight_mapping.get(node, 0)  # 获取权值，如果未在映射中找到则默认为0
        f.write(f"{node}\t{weight}\n")

    # 添加 'IBD_to_normal' 节点，权值为1000
    f.write("IBD_to_normal\t200\n")

# 更新 'unweight.glasso_edge' 文件以添加到 'IBD_to_normal' 的连接
with open(edge_file, 'a') as f:
    for idx in correlation_df.index:
        # 写入到 'IBD_to_normal' 的连接
        f.write("link_{}\t{}\tIBD_to_normal\n".format(link_counter, idx))
        link_counter += 1

# 创建一个ExcelWriter对象
with pd.ExcelWriter('combined_file'+str(key)+'.xlsx', engine='xlsxwriter') as writer:
    # 写入第一个TSV文件到第一个sheet
    df1 = pd.read_csv('unweight.glasso_node'+str(key)+'.tsv', sep='\t')
    df1.to_excel(writer, sheet_name='Sheet1', index=False)

    # 写入第二个TSV文件到第二个sheet
    df2 = pd.read_csv('unweight.glasso_edge'+str(key)+'.tsv', sep='\t')
    df2.to_excel(writer, sheet_name='Sheet2', index=False)